In [20]:
# 1. 기존 라이브러리 강제 삭제
!pip uninstall -y google-generativeai

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5


In [41]:
# 2. 최신 라이브러리 새로 설치
!pip install -q -U google-generativeai

# 확인
print("라이브러리 재설치 완료")

라이브러리 재설치 완료


In [42]:
# 3. 필요한 라이브러리
import google.generativeai as genai # 구글의 Generative AI SDK (google-generativeai) 모델 사용
import json # JSON(JavaScript Object Notation) 형식의 데이터를 다루기 위한 파이썬 내장 모듈
import PIL.Image # Pillow(PIL) 라이브러리의 Image 모듈
from google.colab import files # Google Colab 환경에서 파일 업로드/다운로드를 지원하는 모듈

In [43]:
# 4. API 키 설정
GOOGLE_API_KEY = '' # API 키
genai.configure(api_key=GOOGLE_API_KEY)

In [44]:
# 5. 모델 설정 (가장 안정적인 'gemini-pro-vision' 유지)
model = genai.GenerativeModel('models/gemini-2.5-flash')

# 확인
print(f"모델 '{model.model_name}' 설정 완료")

모델 'models/gemini-2.5-flash' 설정 완료


In [45]:
# 6. 프롬프트
SPORT_data = """
### 역할
너는 스포츠 데이터 분석가야. 이미지에 있는 경기 기록을 분석해서 객관적으로 경기를 요약하고 승부처를 분석해서 지정된 '출력 형식'에 맞춰 경기 핵심 정보를 구조화하는 것이 너의 역할이야.

-----------------------------
### 규칙
함께 제공된 "이미지"를 종합해 다음 필드를 정확히 채워.

- sport:
 - "이미지"가 '축구'인지 '야구'인지 (혹은 '농구' 등 다른 종목인지) 스스로 파악.
- league:
 - 'sport' 필드에서 분류한 종목과 'teams'에 작성된 경기 팀을 조합해서 판단.
 - 없으면 "N/A"로 표기
- teams:
 - 축구의 경우 "홈팀 vs 원정팀" 형식으로 정확히 정규화(Normalization).
 - 야구의 경우 "원정팀 vs 홈팀" 형식으로 정확히 정규화(Normalization).
- stadium:
 - 경기장 이름을 추출하되, 정보가 없으면 "N/A"로 정규화 (Normalization).
- result:
 - "X 대 Y, [승리팀] 승/무" 형식으로만 정확히 정규화(Normalization)
- game_summary:
 - 'sport' 필드의 값에 따라, 해당 종목에 맞는 핵심 스탯을
  (축구: 점유율, 슈팅 / 야구: 안타, 홈런 등) 근거로 하는 분석 리포트를 생성(Inference),
- key_insight:
 - 모든 데이터를 종합하여, 승패를 가른 가장 결정적인 요인을 '선정'하고 그 '이유'를 논리적으로 추론(Inference). (최대 120자)
- 형식:
  - 모든 문자열은 큰따옴표로 감싸고, 내부 큰따옴표는 이스케이프(`\"`)
  - 트레일링 콤마 금지, 키 순서는 자유지만 모든 키 포함
  - 마크다운/설명/코드펜스 출력 금지 (JSON만 출력)

-----------------------------
### Few-shot 예시 (축구)

#### input
{{
  "ocr_text": "경기종료\n홈 강원 0 : 0 전북\n11.01 14:00\n강릉하이원아레나\n강원 VS 전북\n볼점유율 55% 45%\n슈팅 6 5\n유효슈팅 3 1\n코너킥 3 1\n오프사이드 2 3\n선수교체 5 5\n파울 14 15\n경고 2 4\n퇴장 0 0\n[선수 명단 - 강원]\n박상혁 FW 0 0 2 2 4\n[선수 명단 - 전북]\n티아고 FW 0 0 0 0 2"
}}

#### output
{{
  "sport": "축구",
  "league": "K League 1",
  "teams": "강원 vs 전북",
  "stadium": "강릉하이원아레나",
  "result": "0 대 0, 무승부",
  "game_summary": "강원이 55%의 근소한 볼점유율 우위를 가져갔으며, 전체 슈팅(6:5)과 유효슈팅(3:1)에서도 모두 앞서는 공격적인 모습을 보였습니다. 하지만 양 팀 모두 득점에는 실패했으며, 전북이 4장의 경고를 받는 등 다소 거친 경기가 진행되었습니다.",
  "key_insight": "강원이 55%의 점유율과 3배 많은 유효슈팅(3:1)으로 경기를 주도했으나, 양 팀 모두 골 결정력 부족으로 득점 없이 비겼다. (101자)"
}}

-----------------------------
### Few-shot 예시 (야구)

#### input
{{
  "ocr_text": "경기종료\nLG 4 : 1 한화 (홈)\n10.31 18:30\n대전\n승: 켈리허스트 패: 정우주 세: 유영찬\n팀 R H E\nLG 4 11 0\n한화 1 6 1\n안타 11 6\n홈런 0 0\n삼진 9 5\n병살 1 3\n결승타: 오지환(3회 1사 만루서 유격수 희생플라이)\n[선수 명단 - LG]\n켈리허스트 승 7이닝 4피안타 1실점\n[선수 명단 - 한화]\n정우주 패 2이닝 2피안타 1실점"
}}

#### output
{{
  "sport": "야구",
  "league": "KBO 리그",
  "teams": "LG 트윈스 vs 한화 이글스",
  "stadium": "대전",
  "result": "4 대 1, LG 트윈스 승",
  "game_summary": "LG가 11안타를 기록하며 4점을 득점했고, 한화는 6안타 1득점에 그쳤습니다. LG 선발 '켈리허스트'가 7이닝 1실점으로 승리투수가 되었으며, 한화는 1개의 수비 에러를 기록했습니다.",
  "key_insight": "3회초 1사 만루 상황에서 나온 LG '오지환'의 유격수 희생플라이 결승타가 경기의 승패를 결정지었다. (78자)"
}}

-----------------------------
### Input data
OCR에서 추출한 이미지 텍스트: {ocr_text}

-----------------------------
### Output 형식(반드시 JSON만 출력)
{
  "sport": "",
  "league": "",
  "teams": "",
  "stadium": "",
  "result": "",
  "game_summary": "",
  "key_insight":""
}
"""

In [46]:
# 7. 분석할 이미지 업로드
print("분석할 경기의 모든 이미지 (스코어보드, 스탯, 선수 명단 등)를 업로드하세요")
uploaded_files = files.upload()

분석할 경기의 모든 이미지 (스코어보드, 스탯, 선수 명단 등)를 업로드하세요


Saving sample_baseball(MLB)1.png to sample_baseball(MLB)1 (1).png
Saving sample_baseball(MLB)2.png to sample_baseball(MLB)2 (1).png
Saving sample_baseball(MLB)3.png to sample_baseball(MLB)3 (1).png
Saving sample_baseball(MLB)4.png to sample_baseball(MLB)4 (1).png


In [47]:
# 8. 업로드된 이미지를 리스트로 만들기
image_list = []
if len(uploaded_files.keys()) == 0:
    print("이미지가 업로드되지 않았습니다.")
else:
    for file_name in uploaded_files.keys():
        print(f"이미지 '{file_name}' 로드 완료.")
        img = PIL.Image.open(file_name)
        image_list.append(img)

    print(f"\n총 {len(image_list)}장의 이미지가 준비되었습니다.")

이미지 'sample_baseball(MLB)1 (1).png' 로드 완료.
이미지 'sample_baseball(MLB)2 (1).png' 로드 완료.
이미지 'sample_baseball(MLB)3 (1).png' 로드 완료.
이미지 'sample_baseball(MLB)4 (1).png' 로드 완료.

총 4장의 이미지가 준비되었습니다.


In [48]:
# 9. AI 모델 호출 (프롬프트 + 이미지 리스트 함께 전달)
if len(image_list) > 0:
    # AI에게 보낼 최종 리스트 (프롬프트 1개 + 이미지 N개)
    content_to_send = [SPORT_data] + image_list

    print("AI가 여러 장의 이미지를 종합 분석 중...")
    response = model.generate_content(content_to_send)

    print("AI 응답 원본 (JSON 텍스트)")
    print(response.text)

    # 9. JSON 파싱 검증
    # try:
    #     parsed_json = json.loads(response.text)
    #     print("[성공] JSON 파싱 완료!")
    #     print(json.dumps(parsed_json, indent=2, ensure_ascii=False)) # 결과를 예쁘게 출력

    # except Exception as e:
    #     print(f"[실패] JSON 파싱 오류!")
    #     print(f"오류: {e}")
    #     print("AI가 유효하지 않은 JSON을 반환했습니다.")
    try:
          # 1. AI 응답 텍스트에서 첫 '{'를 찾습니다.
          start_index = response.text.find('{')
          # 2. AI 응답 텍스트에서 마지막 '}'를 찾습니다.
          end_index = response.text.rfind('}')

          if start_index != -1 and end_index != -1 and end_index > start_index:
              # 3. '{'부터 '}'까지의 '진짜 JSON' 부분만 잘라냅니다.
              clean_json_text = response.text[start_index : end_index + 1]
          else:
              # '{'나 '}'를 못 찾으면, 그냥 원본 텍스트를 씁니다.
              clean_json_text = response.text
    # 4. 깨끗해진 텍스트로 JSON 파싱을 시도합니다.
          parsed_json = json.loads(clean_json_text)

          print("[성공] JSON 파싱 완료! (청소 성공)")
    # 파싱된 JSON을 예쁘게 출력 (ensure_ascii=False로 한글 유지)
          print(json.dumps(parsed_json, indent=2, ensure_ascii=False))

    except json.JSONDecodeError as e:
          print(f"[실패] JSON 파싱 오류!")
          print(f"오류: {e}")
          print("AI가 유효하지 않은 JSON을 반환했거나, '청소' 코드도 실패했습니다.")
          print("AI 응답 원본 (청소 시도 전)")
          print(response.text) # 디버깅을 위해 원본을 다시 출력
else:
    print("분석할 이미지가 없습니다.")

AI가 여러 장의 이미지를 종합 분석 중...
AI 응답 원본 (JSON 텍스트)
```json
{
  "sport": "야구",
  "league": "MLB",
  "teams": "LA 다저스 vs 토론토",
  "stadium": "로저스 센터",
  "result": "5 대 4, LA 다저스 승",
  "game_summary": "LA 다저스는 11안타 3홈런으로 5득점을 올린 반면, 토론토는 14안타를 기록하고도 1홈런 포함 4득점에 그쳐 다저스가 승리했습니다. 다저스의 스미스, 먼시, 로하스가 각각 홈런을 기록하며 효율적인 공격력을 선보였고, 선발 오타니가 3실점으로 부진했으나 야마모토가 승리투수가 되었습니다. 토론토는 비솃의 홈런에도 불구하고 비버가 패전투수가 되었습니다.",
  "key_insight": "토론토가 14안타로 더 많은 안타를 쳤음에도 불구하고 LA 다저스가 3개의 홈런으로 득점을 효율적으로 연결했으며, 승리투수 야마모토의 2.2이닝 무실점 호투가 승패를 결정지었다. (117자)"
}
```
[성공] JSON 파싱 완료! (청소 성공)
{
  "sport": "야구",
  "league": "MLB",
  "teams": "LA 다저스 vs 토론토",
  "stadium": "로저스 센터",
  "result": "5 대 4, LA 다저스 승",
  "game_summary": "LA 다저스는 11안타 3홈런으로 5득점을 올린 반면, 토론토는 14안타를 기록하고도 1홈런 포함 4득점에 그쳐 다저스가 승리했습니다. 다저스의 스미스, 먼시, 로하스가 각각 홈런을 기록하며 효율적인 공격력을 선보였고, 선발 오타니가 3실점으로 부진했으나 야마모토가 승리투수가 되었습니다. 토론토는 비솃의 홈런에도 불구하고 비버가 패전투수가 되었습니다.",
  "key_insight": "토론토가 14안타로 더 많은 안타를 쳤음에도 불구하고 LA 다저스가 3개의 홈런으로 득점을 효율적으로 연결했으며, 승리투수 야마모토의 2.2이닝 무실점 호투가 승패를 

In [10]:
# v1beta 메뉴판에는 어떤 모델이 있는지 확인하는 디버깅 코드
import google.generativeai as genai

try:
    print(f"API 키로 'v1beta'에서 접근 가능한 모델 목록")

    # API 키가 설정되어 있어야 함 (Cell 1 실행 후)
    # genai.configure(api_key=GOOGLE_API_KEY)

    for m in genai.list_models():
        print(f"- {m.name}")

except Exception as e:
    print(f"모델 목록을 가져오는 데 실패했습니다")
    print(e)

--- 님의 API 키로 'v1beta'에서 접근 가능한 모델 목록 ---
- models/embedding-gecko-001
- models/gemini-2.5-pro-preview-03-25
- models/gemini-2.5-flash-preview-05-20
- models/gemini-2.5-flash
- models/gemini-2.5-flash-lite-preview-06-17
- models/gemini-2.5-pro-preview-05-06
- models/gemini-2.5-pro-preview-06-05
- models/gemini-2.5-pro
- models/gemini-2.0-flash-exp
- models/gemini-2.0-flash
- models/gemini-2.0-flash-001
- models/gemini-2.0-flash-exp-image-generation
- models/gemini-2.0-flash-lite-001
- models/gemini-2.0-flash-lite
- models/gemini-2.0-flash-preview-image-generation
- models/gemini-2.0-flash-lite-preview-02-05
- models/gemini-2.0-flash-lite-preview
- models/gemini-2.0-pro-exp
- models/gemini-2.0-pro-exp-02-05
- models/gemini-exp-1206
- models/gemini-2.0-flash-thinking-exp-01-21
- models/gemini-2.0-flash-thinking-exp
- models/gemini-2.0-flash-thinking-exp-1219
- models/gemini-2.5-flash-preview-tts
- models/gemini-2.5-pro-preview-tts
- models/learnlm-2.0-flash-experimental
- models/gemma-3-